## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#  Import and read the charity_data.csv.
import pandas as pd 
application_dfCOPY = pd.read_csv("../Resources/charity_data.csv")
application_dfCOPY.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
## Let's try to optimize here a bit.
## The status column only has 5 values that are not 1 so that can be dropped as to not overfit
application_df = application_dfCOPY.copy().drop(columns=['EIN', 'NAME', "STATUS"])

In [3]:
# Determine the number of unique values in each column.
## Using .unique is the easiest method of finding unique values so we use a simple for loop to count the values for each column
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

## Let's try 50 as a cutoff point now to see if that improves it.
application_types_to_replace = ["T17", "T29", "T15", "T14", "T25", "T2", "T12"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C8210        1
C2570        1
C4120        1
C2150        1
C2561        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
classif = pd.DataFrame(application_df['CLASSIFICATION'].value_counts())
classifIndexed = classif.reset_index()
classifIndexed

,index,CLASSIFICATION
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
...,...,...
66,C8210,1
67,C2570,1
68,C4120,1
69,C2150,1


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

## Let's try the same new cutoff of 50 again and see how our model changes.
classifications_to_replace = []

for index, row in classifIndexed.iterrows():
  if row["CLASSIFICATION"] < 50:
    classifications_to_replace.append(row["index"])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      391
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

In [9]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,N,5000,1


In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummiedData = pd.get_dummies(application_df)
dummiedData

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,5000,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,5000,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = dummiedData.drop("IS_SUCCESSFUL", axis = 1)

y = dummiedData["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1801) ## Is it bad etiquitte to use personal details as random number seeds?

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
## Isn't this the input layer?

## The last thing I'm going to try is adjusting the units and the number of epochs
nn.add(tf.keras.layers.Dense(units=9, activation="relu", input_dim=54))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

## For our third run, let's try adding a third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 9)                 495       
                                                                 
 dense_11 (Dense)            (None, 15)                150       
                                                                 
 dense_12 (Dense)            (None, 10)                160       
                                                                 
 dense_13 (Dense)            (None, 1)                 11        
                                                                 
Total params: 816
Trainable params: 816
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model

## I found this optimizer code online to help optimize using adam by adjusting the learning rate to decrease loss.
optimized = tf.keras.optimizers.Adam(learning_rate=0.01)
nn.compile(loss="binary_crossentropy", optimizer=optimized, metrics=["accuracy"])

In [30]:
# Train the model
fittedModel = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 1s 682us/step - loss: 0.5528 - accuracy: 0.7319
Epoch 2/150
804/804 [==============================] - 1s 689us/step - loss: 0.5522 - accuracy: 0.7319
Epoch 3/150
804/804 [==============================] - 1s 664us/step - loss: 0.5512 - accuracy: 0.7325
Epoch 4/150
804/804 [==============================] - 1s 667us/step - loss: 0.5514 - accuracy: 0.7333
Epoch 5/150
804/804 [==============================] - 1s 691us/step - loss: 0.5516 - accuracy: 0.7317
Epoch 6/150
804/804 [==============================] - 1s 675us/step - loss: 0.5509 - accuracy: 0.7326
Epoch 7/150
804/804 [==============================] - 1s 654us/step - loss: 0.5503 - accuracy: 0.7330
Epoch 8/150
804/804 [==============================] - 1s 668us/step - loss: 0.5493 - accuracy: 0.7322
Epoch 9/150
804/804 [==============================] - 1s 677us/step - loss: 0.5468 - accuracy: 0.7346
Epoch 10/150
804/804 [==============================] - 1s 677us/step - l

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
## First run. Seems like my model is a bit overfit.
model_loss, model_accuracy = nn.evaluate(X_train_scaled,y_train,verbose=2)
print(f"Training Loss: {model_loss}, Training Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5571 - accuracy: 0.7290 - 225ms/epoch - 841us/step
Loss: 0.5570586919784546, Accuracy: 0.7289795875549316
804/804 - 1s - loss: 0.5404 - accuracy: 0.7393 - 607ms/epoch - 755us/step
Training Loss: 0.540421187877655, Training Accuracy: 0.7393096089363098


In [32]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization-3.h5")